In [1]:
# !pip install transformers -q

     |████████████████████████████████| 778kB 2.7MB/s 
     |████████████████████████████████| 3.0MB 6.9MB/s 
     |████████████████████████████████| 890kB 26.7MB/s 
     |████████████████████████████████| 1.1MB 29.6MB/s 


In [ ]:
# from google.colab import drive
# drive.mount('/gdrive')

In [3]:
# !cp /gdrive/My\ Drive/num_and_eng*.pt .

In [4]:
import transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

In [5]:
tokenizer = AutoTokenizer.from_pretrained("t5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

Some weights of T5ForConditionalGeneration were not initialized from the model checkpoint at t5-small and are newly initialized: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Load model

In [23]:
def load_model(PATH):
    model.load_state_dict(torch.load(PATH, map_location=torch.device('cpu')))
    return model.eval()

In [7]:
def pre_process(input, num2text):
    if num2text is True:
        task = 'number to english'
        input = ' '.join(str(input))
        return f'{task}: {input} </s>'
    else:
        task = 'english to number'
        return f'{task}: {input} </s>'

## Model 1

### Number to English

In [9]:
model = load_model('num_and_eng.pt')

In [13]:
input = pre_process('1234', num2text=True)
input

'number to english: 1 2 3 4 </s>'

In [14]:
encoded_input = tokenizer.encode(input, return_tensors='pt')
encoded_output = model.generate(encoded_input)
output = tokenizer.decode(encoded_output[0])
output

'one hundred twenty-two thousand three hundred forty-four'

### English to Number

In [ ]:
input = pre_process('seven hundred eighty-six thousand six hundred ninety-nine', num2text=False)
input

'english to number: seven hundred eighty-six thousand six hundred ninety-nine </s>'

In [ ]:
encoded_input = tokenizer.encode(input, return_tensors='pt')
encoded_output = model.generate(encoded_input)
output = tokenizer.decode(encoded_output[0])
output = ''.join(output.split(' '))
output

'786699'

## Model 2

### Number to English

In [20]:
model = load_model('num_and_eng_2.pt')

In [36]:
input = pre_process('711', num2text=True)
input

'number to english: 7 1 1 </s>'

In [37]:
encoded_input = tokenizer.encode(input, return_tensors='pt')
encoded_output = model.generate(encoded_input)
output = tokenizer.decode(encoded_output[0])
output

'seven hundred eleven'

### English to Number

In [44]:
input = pre_process('seven hundred eighty-six thousand one', num2text=False)
input

'english to number: seven hundred eighty-six thousand one </s>'

In [45]:
encoded_input = tokenizer.encode(input, return_tensors='pt')
encoded_output = model.generate(encoded_input)
output = tokenizer.decode(encoded_output[0])
output = ''.join(output.split(' '))
output

'786001'

## Model 3

### Number to English

In [46]:
model = load_model('num_and_eng_3.pt')

In [51]:
def extra_pre_process(input, num2text=True):
    max_digit_len = 6
    if len(input) > max_digit_len:
        raise ValueError()

    input = '0' * (max_digit_len - len(''.join(input.split(' ')))) + input
    input = pre_process(input, num2text=num2text)

    return input

In [68]:
input = extra_pre_process('71', num2text=True)
input

'number to english: 0 0 0 0 7 1 </s>'

In [69]:
encoded_input = tokenizer.encode(input, return_tensors='pt')
encoded_output = model.generate(encoded_input)
output = tokenizer.decode(encoded_output[0])
output

'seventy-one'

### English to Number

In [71]:
input = pre_process('eighty-six thousand seventy-three', num2text=False)
input

'english to number: eighty-six thousand seventy-three </s>'

In [74]:
encoded_input = tokenizer.encode(input, return_tensors='pt')
encoded_output = model.generate(encoded_input)
output = tokenizer.decode(encoded_output[0])
output = ''.join(output.split(' '))
output = str(int(output))
output

'86073'

In [73]:
int(output)

86073

## Conclution:

* *Model 3* performs better than others

* *Model 3* has trained on `0`-padded data (lenght-6)
    * `859` -> `000859`
    * `3346` -> `003346`
    * `54` -> `000054`